In [ ]:
import os

!pip install keras4torch
!pip install einops



import  keras4torch
from    keras4torch.callbacks  import ModelCheckpoint,LRScheduler
import  torch
import  torch.nn    as nn
import  torch.optim as optim
import  torch.nn.functional as     F
import  numpy       as np
import  pandas      as pd
from    copy import deepcopy
import  matplotlib.pyplot   as     plt
from    sklearn.preprocessing import StandardScaler, QuantileTransformer
from    datetime import datetime
import  gc
import  STab
from    STab import MyClassLoss,CatMap,Num_Cat,MyRegreLoss
from    STab import mainmodel, LWTA, Gsoftmax
MainModel=mainmodel.MainModel

In [2]:
##Load DAta
X_test_num=pd.DataFrame(np.load('Data/diamond/X_num_test.npy')).astype(np.float64)
X_test_cat=pd.DataFrame(np.load('Data/diamond/X_cat_test.npy'))
y_test=pd.DataFrame(np.load('Data/diamond/Y_test.npy')).astype(np.float64)

X_train_num=pd.DataFrame(np.load('Data/diamond/X_num_train.npy')).astype(np.float64)
X_train_cat=pd.DataFrame(np.load('Data/diamond/X_cat_train.npy'))
Y_train=pd.DataFrame(np.load('Data/diamond/Y_train.npy')).astype(np.float64)


X_valid_num=pd.DataFrame(np.load('Data/diamond/X_num_val.npy')).astype(np.float64)
X_valid_cat=pd.DataFrame(np.load('Data/diamond/X_cat_val.npy'))
y_valid=pd.DataFrame(np.load('Data/diamond/Y_val.npy')).astype(np.float64)

In [3]:
# mapping categorical features to integers
catmap      = CatMap(X_train_cat)
X_test_cat  = catmap(X_test_cat)
X_valid_cat = catmap(X_valid_cat)
X_train_cat = catmap(X_train_cat)

In [4]:
#Normalise In
scalerX = StandardScaler()

scalerX.fit(X_train_num)

X_train_num = scalerX.transform(X_train_num).astype(np.float64)
X_valid_num = scalerX.transform(X_valid_num).astype(np.float64)
X_test_num  = scalerX.transform(X_test_num).astype(np.float64)


In [5]:
#combine numerical and categorical
X_train = np.concatenate([X_train_num,X_train_cat.values],axis=1)
X_test  = np.concatenate([X_test_num,X_test_cat.values],axis=1)
X_valid = np.concatenate([X_valid_num,X_valid_cat.values],axis=1)

In [6]:
def True_MSE(p,t):
    return np.mean( np.square(p-t))

In [7]:
checkpoint='saved/savefileDI'

In [ ]:

Or_model = MainModel(
    categories        = (5,7,8,),
    num_continuous    = 6,
    dim               = 96  ,
    dim_out           = 1,
    depth             = 4,
    heads             = 8,
    attn_dropout      = 0.1 ,
    ff_dropout        = 0.1,
    U                 = 2,
    cases             = 16

)
no_model = Num_Cat(Or_model,Sample_size=16)
model    = keras4torch.Model(no_model,).build([9])

In [ ]:


#Warm Up
optimizer=torch.optim.AdamW(model.parameters(),lr=0.0001,weight_decay=0.0001,)
sch=torch.optim.lr_scheduler.LinearLR(optimizer, start_factor=0.0001, total_iters=5,  verbose=False)
model.compile(optimizer=optimizer, loss=MyRegreLoss(0.01,1), metrics=['mse'])
callbacks=[LRScheduler(sch)]
model.fit(X_train, Y_train.values,
        epochs=5, batch_size=512,
        validation_data=(X_valid,y_valid.values),
        verbose=2,validation_batch_size=1024,
        callbacks=callbacks)

#Main Train
optimizer=torch.optim.AdamW(model.parameters(),lr=0.001,weight_decay=0.0001,)
model.compile(optimizer=optimizer, loss=MyRegreLoss(0.01,1), metrics=['mse'])
scheduler =LRScheduler( torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=5, factor=0.5,min_lr=0.000001))
callbacks=[ModelCheckpoint(checkpoint,monitor='val_mse',mode='min'),scheduler]
model.fit(X_train, Y_train.values,
        epochs=80, batch_size=512,
        validation_data=(X_valid,y_valid.values),
        verbose=2,validation_batch_size=1024,
        callbacks=callbacks)


model.load_weights(checkpoint)


logits=pd.DataFrame()

for i in range(0,64):

        logits[i]=pd.DataFrame(model.predict(X_test,batch_size=256))



Test = (True_MSE(logits.mean(axis=1).values.reshape((-1)),y_test.values.reshape((-1))))



print('Test Score:',Test)
